# T1-Mapping with DESPOT1 / VFA

This notebook deomstrates how T1 mapping with a basic `nipype` pipeline with `QUIT` programs.

It has two direct dependencies - `qipype` and `nanslice`, both of which are available on PyPI. Install with `pip install qipype nanslice`.

This pipeline (http://github.com/spinicist/nanslice) downloads the BrainWeb brain & B1 phantoms from http://brainweb.bic.mni.mcgill.ca. It then replaces the tissue classification labels with values of Proton Density and T1, simulates an SPGR/FLASH image with some added noise, and finally uses that simulated data to fit for T1 and PD again.

## Imports

In [ ]:
%matplotlib inline
from qipype.interfaces.relax import DESPOT1, DESPOT1Sim
from qipype.sims import init_brainweb, make_phantom
import nanslice.jupyter as ns
import nibabel as nib

## Create Phantom Data

This section downloads the Brainweb phantoms. These are stored MINC format so we load the images using `nibabel` to get the raw data later.

# Data Fetching

We now download the BrainWeb brain & T1 phantoms, if they are not already present in the current directory. QUIT provides a utility function for this.

In [ ]:
init_brainweb()

Now we use another QUIT function to convert the BrainWeb phantoms (which are tissue class maps) into parameter maps, in this case Proton Density and T1. There are 10 tissue classes:

0. Background
1. CSF
2. Grey Matter
3. White Matter
4. Fat
5. Muscle/Skin
6. Skin
7. Skull
8. Glial Matter
9. Connective

The `make_phantom` function requires a dictionary, where every parameter map we want to make is a key, and the value is a 10 element array with the values for each class. To keep things simple here, we only specify values for CSF, GM & WM, and set the other tissue types to zero. We also specify that we want a mask and a B1 map for later use, and sub-sample the images by a factor of 2 to make it faster.

In [ ]:
param_dict = {'PD': [0, 1, 0.8, 0.7, 0, 0, 0, 0, 0, 0],
              'T1': [0, 3.0, 1.3, 0.9, 0, 0, 0, 0, 0, 0]}
mask = [0, 1, 1, 1, 0, 0, 0, 0, 0, 0]
make_phantom(param_dict, mask=mask, B1=True, subsamp=2)
display(ns.three_plane('T1.nii.gz'))
display(ns.three_plane('B1.nii.gz'))

## Simulate Image

As well as fitting image data to find parameters, QUIT allows you to simulate the images from the paramters. Here we use the phantom images from the step above to create some simple gradient-echo (also called FLASH, SPGR or FFE depending on vendor) images.

First we define a dictionary that sets the sequence parameters for the type of scan we are simulating. In this case we only need the TR and the flip-angle (FA). The TR is specified in seconds, not milliseconds, and there are multiple values for the flip-angle as we need to simulate multiple images.

We then define and run the simulator. This will return a results object that contains the paths to the output files, which we use to display both volumes in the output image. The noise is defined as an SNR level relative assuming a nominal PD of 1 (this is why the PD of CSF was set to 1 above).

In [ ]:
sequence_parameters = {'SPGR': {'TR': 10e-3, 'FA': [3, 18]}}
simulator_results = DESPOT1Sim(sequence=sequence_parameters, out_file='sim_spgr.nii.gz', noise=0.001, PD_map='PD.nii.gz', T1_map='T1.nii.gz').run()

display(ns.three_plane(simulator_results.outputs.out_file, volume=0))
display(ns.three_plane(simulator_results.outputs.out_file, volume=1))

## Fit Data and Compare to Reference

Now we have simulated data we can run the DESPOT1/VFA fitting code and compare the results to our phantom reference data. First we fit without a mask, which produces an annoying noisy background where we had no signal. Then we fit again, specifying a mask.

We need to specify a `prefix` argument to one of the fitting calls to stop the output files being overwritten. QUIT programs will produce a set of output files of the form `{prefix}{method}_{parameter}`. In this case the `method` is `D1`, short for DESPOT1, and the parameters will be `PD` and `T1`. It is unusual to run a QUIT program more than once in the same directory, but if you do make sure to specify a prefix value.

In [ ]:
fit_no_mask = DESPOT1(sequence=sequence_parameters, in_file=simulator_results.outputs.out_file, prefix='no_mask')
no_mask_results = fit_no_mask.run()
fit_with_mask = DESPOT1(sequence=sequence_parameters, in_file=simulator_results.outputs.out_file, mask_file='mask.nii.gz')
mask_results = fit_with_mask.run()

display(ns.compare('T1.nii.gz', no_mask_results.outputs.T1_map, title='Unmasked T1 Comparison'))
display(ns.compare('T1.nii.gz', mask_results.outputs.T1_map, title='Masked T1 Comparison'))
display(ns.compare('PD.nii.gz', mask_results.outputs.PD_map, title='Masked PD Comparison'))

## B1 Mapping

SPGR images are affected by B1 inhomogeneity. In the above simulation, we did not specify a B1 map and assumed that B1 was flat, but this is not true in real images. We will now add B1 into both our simulation and fitting.

For DESPOT1, and most other methods, B1 is a "fixed" parameter, in contrast to T1 & PD which are "varying" parameters. This means that DESPOT1 does not fit a value for B1, it must be measured with another scan and is then fixed within each voxel. There are two methods in QUIT for calculating B1 maps (see https://quit.readthedocs.io/en/latest/Docs/B1.html).

First we simulate our input images again, supplying the B1 map, but fit without supplying the B1 map. We add the `prefix` parameter to the fit to avoid overwriting the output of the previous fitting run. We set the window to a narrow range to emphasise the B1 inhomogeneity in grey matter.

In [ ]:
b1_sim_results = DESPOT1Sim(sequence=sequence_parameters, out_file='sim_b1.nii.gz', B1_map='B1.nii.gz', noise=0.001, PD_map='PD.nii.gz', T1_map='T1.nii.gz').run()
no_b1_fit_results = DESPOT1(prefix='no_b1', mask_file='mask.nii.gz', sequence=sequence_parameters, in_file=b1_sim_results.outputs.out_file).run()
display(ns.compare('T1.nii.gz', no_b1_fit_results.outputs.T1_map, clim=(0.8,1.5), title='Phantom Vs No-B1 in Fitting'))

Finally, we fit again, this time supplying the B1 map, and compare the results both to the incorrect fit without B1 and with the original fit.

In [ ]:
b1_fit_results = DESPOT1(prefix='b1', sequence=sequence_parameters, mask_file='mask.nii.gz', in_file=b1_sim_results.outputs.out_file, B1_map='B1.nii.gz').run()
display(ns.compare(b1_fit_results.outputs.T1_map, no_b1_fit_results.outputs.T1_map, clim=(0.8, 1.5), title='B1 fit versus no B1'))
display(ns.compare(b1_fit_results.outputs.T1_map, mask_results.outputs.T1_map, clim=(0.8, 1.5), title='B1 fit versus original'))